In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import CM4Xutils
CM4Xutils.__version__

'0.5.0'

In [3]:
import xarray as xr
import numpy as np
import cftime

In [4]:
ds = xr.open_dataset("../data/coarsened/CM4Xp25_budgets_sigma2_2000-2004.zarr")
ds = ds.sel(exp="forced").drop_vars(["RAIN", "EVAP", "LSNK", "LSRC", "SNOWFL", "ePBL_h_ML", "zos", "siconc", "sithick", "taux", "tauy"]).drop_vars(["exp"])

/work/hfd/.conda/envs/CM4X/lib/python3.12/site-packages/xarray/backends/plugins.py:159: RuntimeWarning: 'netcdf4' fails while guessing
  warnings.warn(f"{engine!r} fails while guessing", RuntimeWarning)
/work/hfd/.conda/envs/CM4X/lib/python3.12/site-packages/xarray/backends/plugins.py:159: RuntimeWarning: 'scipy' fails while guessing
  warnings.warn(f"{engine!r} fails while guessing", RuntimeWarning)


### Compute annual mean

In [5]:
def to_numeric_time(date):
    # Convert to number of days since 0001-01-01
    return cftime.date2num(date, units='days since 0001-01-01', calendar='noleap')

# Select just the first year's worth of data
ds = ds.isel(time=np.arange(0, 12, 1), time_bounds=np.arange(0, 13, 1))

# Apply the function to your time array
numeric_time = np.vectorize(to_numeric_time)(ds.time_bounds.values)

# Convert days to seconds
seconds_array = numeric_time * 24 * 60 * 60

ds = ds.assign_coords({"time_delta":xr.DataArray(np.diff(seconds_array), dims=("time",))})

In [6]:
# Annual-mean, weighted by length of each month
# Note: da.weighted behaves strangely when there are NaNs. Fill the NaNs for correct behavior!
ds_mean = ds.fillna(0.).drop_dims("time_bounds").weighted(ds.time_delta).mean(dim="time")
ds_mean = ds_mean.expand_dims(["time"]).assign_coords({"time": xr.DataArray(ds.time_bounds[[6]].values, dims=("time",))})

# Keep only the snapshots at the beginning and end of the year
ds_snaps = ds.fillna(0.).isel(time_bounds=np.array([0,-1])).drop_dims("time")

### Save density-binned and coarsened budgets

In [7]:
ds_example = xr.merge([ds_snaps, ds_mean])
ds_example.attrs = {"description": "An example netcdf file containing all variables required to close mass, heat, salt, and thermodynamic density budgets in MOM6. All variables have been conservatively coarsened from an higher resolution (eddy-permitting) native grid."}

In [8]:
ds_example.to_netcdf("../data/MOM6_global_example_sigma2_budgets.nc", mode="w")

### Save water mass transformation budget in density coordinates

In [9]:
import CM4Xutils
grid = CM4Xutils.ds_to_grid(ds_example)

Inferring Z grid coordinate: density `sigma2`


In [10]:
import warnings
import xbudget
import xwmb
with warnings.catch_warnings():
    warnings.simplefilter(action='ignore', category=FutureWarning)

    budgets_dict = xbudget.load_preset_budget(model="MOM6_3Donly")
    xbudget.collect_budgets(grid, budgets_dict)
    
    wmb = xwmb.WaterMassBudget(
        grid,
        budgets_dict,
        method="xgcm"
    )
    wmb.mass_budget("sigma2", greater_than=True, integrate=True)
    wmt = wmb.wmt
    wmt.load()

In [11]:
wmt.drop_dims(["xh", "yh"]).to_netcdf("../data/MOM6_global_example_sigma2_wmb.nc", mode="w")